<a href="https://colab.research.google.com/github/TusharMohan124/GuppyGuru/blob/master/BestModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 40.9 MB/s eta 0:00:00
  Attempting uninstall: autograd
    Found existing installation: autograd 1.6.2
    Uninstalling autograd-1.6.2:
      Successfully uninstalled autograd-1.6.2


In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer

from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import math


num_qubits = 12
num_layers = 3

dev = qml.device("default.qubit", wires=num_qubits)

# quantum circuit functions
def statepreparation(x):
    qml.AngleEmbedding(x, wires=range(0, num_qubits))

def layer(W):

    for i in range(num_qubits):
      qml.Rot(W[i, 0], W[i, 1], W[i, 2], wires=i)

    for i in range(num_qubits):
      qml.CNOT(wires=[i % num_qubits, (i+1) % num_qubits])

@qml.qnode(dev, interface="autograd")
def circuit(weights, x):

    statepreparation(x)

    for W in weights:
        layer(W)

    return qml.expval(qml.PauliZ(0))

def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

def accuracy(labels, predictions):

    acc = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < .5:
            acc = acc + 1
    acc = acc / len(labels)

    return acc

def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)



# preparing data
df_train = pd.read_csv('forklift_16features_1to3weeks_balanced.csv')
df_filtered = df_train[df_train['asset_engine_hours'].notna()]
df_train = df_filtered.sample(frac=1, random_state=42)[:200]




cols_model = df_train.iloc[:, 1:13]
cols_model1 = df_train.iloc[:, 1:9]
cols_model2 = df_train.iloc[:, 11:15]


for i in range(num_qubits):
  if i < 8:
    cols_model.iloc[:,i] = cols_model1.iloc[:,i]
  else:
    cols_model.iloc[:,i] = cols_model2.iloc[:,i-8]

print(f'Data: {cols_model}')



X_train, X_test, Y_train, Y_test = train_test_split(cols_model, df_train['needs_maintenance_before_routine'], test_size=0.10, random_state=42, stratify=df_train['needs_maintenance_before_routine'])

X_train = np.array(X_train.values, requires_grad=False)
Y_train = np.array(Y_train.values, requires_grad=False)


# setting init params

np.random.seed(0)
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

opt = AdamOptimizer(0.125)
num_it = 500
batch_size = 200

weights = weights_init
bias = bias_init
for it in range(num_it):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = Y_train[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute accuracy
    predictions = [variational_classifier(weights, bias, x) for x in X_train]
    acc = accuracy(Y_train, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f}".format(
            it + 1, cost(weights, bias, X_train, Y_train), acc
        )
    )
    g1 = 0.5*np.linalg.norm(np.cumsum(np.array(opt.fm[0]))) + 0.5*np.linalg.norm(np.cumsum(np.array(opt.fm[1])))
    print(f'Grad: {g1}')

X_test = np.array(X_test.values, requires_grad=False)
Y_test = np.array(Y_test.values, requires_grad=False)

predictions = [variational_classifier(weights, bias, x) for x in X_test]

accuracy_score(Y_test, predictions)
precision_score(Y_test, predictions)
recall_score(Y_test, predictions)
f1_score(Y_test, predictions, average='macro')

<ipython-input-19-b2384748c840>:87: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  cols_model.iloc[:,i] = cols_model2.iloc[:,i-8]


Data:        asset_department  asset_energy_type  asset_manufacturer  asset_model  \
15976                 3                  0                   2            1   
23638                 3                  0                   2            1   
19968                 1                  1                   0            4   
21642                 6                  3                   1            0   
7217                  1                  1                   0            4   
...                 ...                ...                 ...          ...   
2325                  6                  0                   0            3   
2981                  6                  3                   1            0   
4990                  4                  3                   0            7   
22162                 3                  0                   2            1   
19208                 6                  3                   1            0   

       asset_status_code  location_airport_co

KeyboardInterrupt: ignored